In [74]:
%pip install langchain-text-splitters transformers pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [75]:
import numpy as np
import pandas as pd

In [76]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

# Load a tokenizer for a BERT-like model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def count_tokens(text):
    return len(tokenizer.encode(text))

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
    # length_function=len,
    length_function=count_tokens,
    # is_separator_regex=False,
)

In [77]:
def chunk(text):
    texts = text_splitter.create_documents([text])
    chunks = []
    for i,t in enumerate(texts):
        # replace newlines with spaces this can help keep word boundires
        chunks.append(t.page_content.replace("\n", " "))
    return chunks

In [78]:
df = pd.read_csv('./datasets/text_classes.csv', delimiter='|')
# This shuffles the dataframe, but in a non-reproducable way
# df = df.sample(frac=1).reset_index(drop=True)
# Instead shuffle dataframe, but in a reproducable way
SEED=42
np.random.seed(SEED)
idx = df.index.to_list()
np.random.shuffle(idx)
res = df.loc[idx].reset_index(drop=True)
# 4353
# display(df)

In [79]:
from tables import classes, schema_dicts
print(classes)
print(schema_dicts)

['Intellectual Developmental Disorders', 'Communication Disorders', 'Autism Spectrum Disorder', 'Attention-Deficit/Hyperactivity Disorder', 'Specific Learning Disorder', 'Motor Disorders', 'Other Neurodevelo mental Disorders p', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Catatonia', 'Bipolar and Related Disorders', 'Depressive Disorders', 'Anxiety Disorders', 'Obsessive-Compulsive and Related Disorders', 'Trauma- and Stressor-Related Disorders', 'Dissociative Disorders', 'Somatic Symptom and Related Disorders', 'Feeding and Eating Disorders', 'Elimination Disorders', 'Sleep-Wake Disorders', 'Breathing-Related Sleep Disorders', 'Parasomnias', 'Sexual Dysfunctions', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct Disorders', 'Neurocognitive Disorders', 'Personality Disorders', 'Cluster A Personality Disorders', 'Cluster B Personality Disorders', 'Cluster C Personalit Disorders y', 'Other Personality Disorders', 'Paraphilic Disorders', 'Other Mental Disorders a

In [ ]:
schema_index = 0
max_avg_distance = 20

In [81]:
import weaviate
import weaviate.classes as wvc

def query_top_x(text, limit=1, max_distance=20):
    client = weaviate.connect_to_local()
    res = None
    try:
        collection = client.collections.use(schema_dicts[schema_index]["schema"])

        results = collection.query.near_text(
            # Because near_text is a purely vector search, we get a
            # distance and no score
            return_metadata=wvc.query.MetadataQuery(
                score=True, explain_score=True, distance=True, certainty=True
            ),
            query=text,
            limit=limit
        )
        for r in results.objects:
            if r.metadata.distance < max_distance:
                res = [classes.index(r.properties['title']), r.metadata.distance]

        return res
    finally:
        client.close()

In [82]:
# item = 1

# chart_chunks = chunk(df.iloc[item]["chart_text"])
# print(f"Chunks size: {len(chart_chunks)}")

# scores = []
# for c in chart_chunks:
#     d = query_top_five(c)
#     if d is not None:
#         scores.append(d)

# print(f"Scores size: {len(scores)}")

In [83]:
def score_document(s, max_dist=10):
    cat = {}
    top_results = []

    for i, _ in enumerate(classes):
        cat[i] = {"count":0, "distance":0}

    for _, d in enumerate(s):
        cat[d[0]]["count"] += 1
        cat[d[0]]["distance"] += d[1]
        cat[d[0]]["avg"] = cat[d[0]]["distance"] / cat[d[0]]["count"]

    for i, _ in enumerate(classes):
        if cat[i]["count"] > 0 and cat[i]["avg"] < max_dist:
            top_results.append([cat[i]["avg"], i])

    return top_results

# top = np.array(score_document(scores, max_avg_distance))
# print(np.sort(top, axis=0))
# print('--------------')
# print(f'{df.iloc[item]["class_number"]:02} {classes[df.iloc[item]["class_number"]]}')

In [84]:
test_range = 30

results = []
for i in range(test_range): # 4353//4):
    chart_chunks = chunk(df.iloc[i]["chart_text"])
    print(f"Chunks size: {len(chart_chunks)}")

    scores = []
    for c in chart_chunks:
        d = query_top_x(c, 5)
        if d is not None:
            scores.append(d)

    top = np.array(score_document(scores, max_avg_distance))
    top = np.sort(top, axis=0)

    results.append([i, df.iloc[i]["class_number"], top])

Chunks size: 12
Chunks size: 45


Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Created a chunk of size 546, which is longer than the specified 400


Chunks size: 11
Chunks size: 14
Chunks size: 11
Chunks size: 4
Chunks size: 1090
Chunks size: 4
Chunks size: 6
Chunks size: 7
Chunks size: 74
Chunks size: 4
Chunks size: 7
Chunks size: 313
Chunks size: 21


Created a chunk of size 466, which is longer than the specified 400


Chunks size: 20
Chunks size: 13
Chunks size: 8
Chunks size: 26
Chunks size: 72
Chunks size: 40
Chunks size: 2
Chunks size: 4
Chunks size: 90
Chunks size: 140
Chunks size: 4
Chunks size: 14
Chunks size: 18
Chunks size: 7
Chunks size: 11


## Scoring

Correct answer in the top 5

In [85]:
df_results = pd.DataFrame(results, columns=["record", "original", "predict"])

def top5(x):
    return np.array([y[1] for i,y in enumerate(x) if i < 5])

def in_top_5(orig, top5):
    return 1 if orig in top5 else 0

df_results['top5'] = df_results['predict'].map(top5)
df_results['inTop5'] = df_results.apply(lambda x: in_top_5(orig=x['original'], top5=x['top5']), axis=1)

print(f"In the top 5 {df_results['inTop5'].sum() / test_range}")
display(df_results)

In the top 5 0.4


,record,original,predict,top5,inTop5
0,0,10,"[[8.181249618530273, 8.0], [8.702366828918457,...","[8.0, 9.0, 10.0, 17.0, 18.0]",1
1,1,10,"[[8.370892471737331, 8.0], [8.758625030517578,...","[8.0, 9.0, 10.0, 16.0, 18.0]",1
2,2,11,"[[7.455981254577637, 9.0], [8.333358764648438,...","[9.0, 10.0, 11.0, 18.0, 20.0]",1
3,3,11,"[[7.941040992736816, 8.0], [7.971506118774414,...","[8.0, 9.0, 10.0, 16.0, 18.0]",0
4,4,10,"[[7.701631546020508, 8.0], [7.934941291809082,...","[8.0, 9.0, 10.0, 16.0, 18.0]",1
5,5,11,"[[7.643545150756836, 9.0], [9.6237154006958, 2...","[9.0, 23.0, 24.0, 28.0]",0
6,6,10,"[[9.674255395546938, 3.0], [9.73230960152366, ...","[3.0, 5.0, 8.0, 9.0, 10.0]",1
7,7,10,"[[10.829326629638672, 16.0], [11.0590038299560...","[16.0, 18.0, 24.0]",0
8,8,10,"[[7.878737449645996, 10.0], [8.006170272827148...","[10.0, 16.0, 17.0, 18.0, 23.0]",1
9,9,11,"[[7.072710990905762, 8.0], [8.568474292755127,...","[8.0, 9.0, 10.0, 16.0, 20.0]",0


### Mean Reciprocal Rank (MRR)

The position of the first correct result is important

For each query, find the rank of the first correct result. The Reciprocal Rank (RR) is $1 / rank$. MRR is the average RR across all queries.

Example: Suppose you have 3 queries:
- Query 1: Correct result is ranked 1st → RR = 1/1 = 1.0
- Query 2: Correct result is ranked 3rd → RR = 1/3 ≈ 0.33
- Query 3: Correct result is ranked 2nd → RR = 1/2 = 0.5

$$
MRR = (1.0 + 0.33 + 0.5) / 3 ≈ 0.61
$$

$$
MRR = \frac{1}{∣Q∣} \sum_{i=1}^{∣Q∣}\frac{​1}{rank_i}
$$

A good Mean Reciprocal Rank (MRR) value typically ranges from 0.6 to 1.0

In [86]:
def mmr(orig, top5):
    idx = np.where(top5 == orig)[0]
    return (1/(idx+1) if idx.size != 0 else [0])[0]

df_results['mmr'] = df_results.apply(lambda x: mmr(orig=x['original'], top5=x['top5']), axis=1)

print(f"MMR: {df_results['mmr'].sum() / test_range}")
display(df_results)

MMR: 0.15388888888888888


,record,original,predict,top5,inTop5,mmr
0,0,10,"[[8.181249618530273, 8.0], [8.702366828918457,...","[8.0, 9.0, 10.0, 17.0, 18.0]",1,0.333333
1,1,10,"[[8.370892471737331, 8.0], [8.758625030517578,...","[8.0, 9.0, 10.0, 16.0, 18.0]",1,0.333333
2,2,11,"[[7.455981254577637, 9.0], [8.333358764648438,...","[9.0, 10.0, 11.0, 18.0, 20.0]",1,0.333333
3,3,11,"[[7.941040992736816, 8.0], [7.971506118774414,...","[8.0, 9.0, 10.0, 16.0, 18.0]",0,0.000000
4,4,10,"[[7.701631546020508, 8.0], [7.934941291809082,...","[8.0, 9.0, 10.0, 16.0, 18.0]",1,0.333333
5,5,11,"[[7.643545150756836, 9.0], [9.6237154006958, 2...","[9.0, 23.0, 24.0, 28.0]",0,0.000000
6,6,10,"[[9.674255395546938, 3.0], [9.73230960152366, ...","[3.0, 5.0, 8.0, 9.0, 10.0]",1,0.200000
7,7,10,"[[10.829326629638672, 16.0], [11.0590038299560...","[16.0, 18.0, 24.0]",0,0.000000
8,8,10,"[[7.878737449645996, 10.0], [8.006170272827148...","[10.0, 16.0, 17.0, 18.0, 23.0]",1,1.000000
9,9,11,"[[7.072710990905762, 8.0], [8.568474292755127,...","[8.0, 9.0, 10.0, 16.0, 20.0]",0,0.000000
